## 模型导出

使用LoRA 适配器训练好模型后，每次推理时候都需要分别加载基座模型和 LoRA 适配器，一方面比较繁琐，另一方面也不利于模型部署，因此有必要将基座模型和 LoRA 适配器进行合并，导出成一个模型。

LLamaFactory中自带模型合并的功能，只需要从`examples/merge_lora/llama3_lora_sft.yaml`拷贝一份配置根据自己的情况进行修改。修改后的配置文件示例如下：

In [9]:
!cat /data2/downloads/LLaMA-Factory/merge_qwen2_lora_sft.yaml

### Note: DO NOT use quantized model or quantization_bit when merging lora adapters

### model
model_name_or_path: /data2/anti_fraud/models/modelscope/hub/Qwen/Qwen2-1___5B-Instruct
adapter_name_or_path: /data2/anti_fraud/models/Qwen2-1___5B-Instruct_ft_0830_2/checkpoint-1900
template: qwen
finetuning_type: lora

### export
export_dir: /data2/anti_fraud/models/Qwen2-1__5B-Instruct-anti_fraud_1__0
export_size: 5
export_device: cpu
export_legacy_format: false


- export_size: 指定导出的模型文件分片大小，单位为GB, 适用于模型比较大需要分片导出的场景，以便在内存限制的设备上加载。
- export_device: 导出模型时使用的设备。可以选择 cpu 或 cuda（即 GPU）。如果有强大的 GPU 可以使用，选择 cuda 可以加快导出速度。
- export_legacy_format: 指定是否使用旧格式导出模型。通常情况下，选择 false 以导出使用最新格式的模型。

In [4]:
使用`llamafactory-cli`执行export命令导出模型。

SyntaxError: invalid character '。' (U+3002) (2397837049.py, line 1)

In [14]:
!llamafactory-cli export /data2/downloads/LLaMA-Factory/merge_qwen2_lora_sft.yaml

[INFO|tokenization_utils_base.py:2048] 2025-03-16 19:42:15,767 >> loading file vocab.json
[INFO|tokenization_utils_base.py:2048] 2025-03-16 19:42:15,768 >> loading file merges.txt
[INFO|tokenization_utils_base.py:2048] 2025-03-16 19:42:15,768 >> loading file tokenizer.json
[INFO|tokenization_utils_base.py:2048] 2025-03-16 19:42:15,768 >> loading file added_tokens.json
[INFO|tokenization_utils_base.py:2048] 2025-03-16 19:42:15,768 >> loading file special_tokens_map.json
[INFO|tokenization_utils_base.py:2048] 2025-03-16 19:42:15,768 >> loading file tokenizer_config.json
[INFO|tokenization_utils_base.py:2048] 2025-03-16 19:42:15,768 >> loading file chat_template.jinja
[INFO|tokenization_utils_base.py:2313] 2025-03-16 19:42:16,100 >> Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
[INFO|configuration_utils.py:697] 2025-03-16 19:42:16,103 >> loading configuration file /data2/anti_fraud/models/modelscope/hub/Qwen/Qwen2-1__

In [ ]:
查看导出的模型文件，可以看到已经按照我们的配置将模型参数进行了两个分片。

In [15]:
!ls -l /data2/anti_fraud/models/Qwen2-1__5B-Instruct-anti_fraud_1__0

total 3030606
-rw-r--r-- 1 root root        424 Mar 16 19:43 Modelfile
-rw-r--r-- 1 root root         80 Mar 16 19:43 added_tokens.json
-rw-r--r-- 1 root root        773 Mar 16 19:42 config.json
-rw-r--r-- 1 root root        242 Mar 16 19:42 generation_config.json
-rw-r--r-- 1 root root    1671853 Mar 16 19:43 merges.txt
-rw-r--r-- 1 root root 3087467144 Mar 16 19:43 model.safetensors
-rw-r--r-- 1 root root        367 Mar 16 19:43 special_tokens_map.json
-rw-r--r-- 1 root root   11418266 Mar 16 19:43 tokenizer.json
-rw-r--r-- 1 root root       1355 Mar 16 19:43 tokenizer_config.json
-rw-r--r-- 1 root root    2776833 Mar 16 19:43 vocab.json


In [16]:
!ls -l /data2/anti_fraud/models/modelscope/hub/Qwen/Qwen2-1___5B-Instruct

total 3026332
-rw-r--r-- 1 root root      11344 Mar 15 23:23 LICENSE
-rw-r--r-- 1 root root       3537 Mar 15 23:23 README.md
-rw-r--r-- 1 root root        660 Mar 15 23:23 config.json
-rw-r--r-- 1 root root         48 Mar 15 23:23 configuration.json
-rw-r--r-- 1 root root        242 Mar 15 23:23 generation_config.json
-rw-r--r-- 1 root root    1671839 Mar 15 23:23 merges.txt
-rw-r--r-- 1 root root 3087467144 Mar 15 23:27 model.safetensors
-rw-r--r-- 1 root root    7028015 Mar 15 23:23 tokenizer.json
-rw-r--r-- 1 root root       1287 Mar 15 23:23 tokenizer_config.json
-rw-r--r-- 1 root root    2776833 Mar 15 23:23 vocab.json


## 测试模型性能

In [17]:
%run evaluate.py
model_path = '/data2/anti_fraud/models/Qwen2-1__5B-Instruct-anti_fraud_1__0'

device = 'cuda'

In [18]:
testdata_path = '/data2/anti_fraud/dataset/eval0819.jsonl'
evaluate(model_path, '', testdata_path, device, batch=True, debug=True)

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.
progress: 100%|██████████| 2348/2348 [01:21<00:00, 28.84it/s]

tn：1147, fp:18, fn:202, tp:981
precision: 0.9819819819819819, recall: 0.8292476754015216, accuracy: 0.9063032367972743


In [19]:

testdata_path = '/data2/anti_fraud/dataset/test0819.jsonl'
evaluate(model_path, '', testdata_path, device, batch=True, debug=True)

progress: 100%|██████████| 2349/2349 [01:19<00:00, 29.53it/s]

tn：1153, fp:14, fn:221, tp:961
precision: 0.9856410256410256, recall: 0.8130287648054145, accuracy: 0.8999574286930608


## 模型部署

查看配置文件：

In [20]:
!less /data2/downloads/LLaMA-Factory/qwen2_inference.yaml

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/bin/bash: line 1: less: command not found


通过llamafactory启动推理服务的方式：
```
llamafactory-cli api /data2/downloads/LLaMA-Factory/qwen2_inference.yaml
```
为了收集日志及指定环境变量，封装为一个启动脚本。

In [ ]:
!less ~/anti_fraud_start.sh

启动服务后，使用openai的api来测试服务功能。

## 推理测试

In [21]:
%%time

# api_call_example.py
from openai import OpenAI

def predict(prompt):
    client = OpenAI(api_key="0",base_url="http://127.0.0.1:8000/v1")
    messages = [{"role": "user", "content": prompt}]
    result = client.chat.completions.create(messages=messages, model="Qwen2-1__5B-Instruct")
    return result.choices[0].message.content

predict("详细介绍下你自己。")

CPU times: user 620 ms, sys: 83.5 ms, total: 704 ms
Wall time: 2.36 s


'我是阿里云开发的一种超大规模语言模型，我的名字是通义千问。我可以通过理解并生成人类自然语言与用户进行交互，提供帮助和解答问题。我能够进行多领域的知识学习，涵盖历史、科学、文化、艺术等各个领域，拥有丰富的知识库和算法模型。我还可以进行文本生成、问答、代码生成等多种任务，为用户提供高效、准确、个性化的服务。我不断学习和进步，致力于成为用户的智能助手和知识伙伴。'

In [22]:
def build_fraud_prompt(content):
    return f"下面是一段对话文本, 请分析对话内容是否有诈骗风险，以json格式输出你的判断结果(is_fraud: true\/false)。\n{content}"

<>:2: SyntaxWarning: invalid escape sequence '\/'
<>:2: SyntaxWarning: invalid escape sequence '\/'
/tmp/ipykernel_2436/1171486641.py:2: SyntaxWarning: invalid escape sequence '\/'
  return f"下面是一段对话文本, 请分析对话内容是否有诈骗风险，以json格式输出你的判断结果(is_fraud: true\/false)。\n{content}"


In [35]:
%%time
content = '小明: 我们有专业的团队进行风险管理，可以确保你的投资安全。而且我们会提供实时的投资动态，让你随时掌握投资情况。\n小红: 那我什么时候能回收投资并获得收益呢？\n小明: 投资期限为1年，到期后你就可以回收本金和收益。\n小红: 听起来还不错，我会考虑一下。谢谢你的介绍。\n小明: 不客气，如果你有任何问题，随时可以问我。希望你能抓住这个机会，获得更多的财富。'
predict(build_fraud_prompt(content))

CPU times: user 52.5 ms, sys: 21 μs, total: 52.6 ms
Wall time: 199 ms


'{"is_fraud": true}'

In [31]:
%%time
content = '发言人3: 然后从不管是分业务板块的一个营收占比和分地区板块的或分地区的一个营收占比来看的话，首先这个屠宰基本上是占了公司总体营收的90%以上的一个比例，但是我们可以看到左下角的这个图，近几年畜禽养殖板块其实更多就来自于生猪养殖板块，它对于整个营收的一个占比它其实有一个明显的提升，而且随着未来公司出栏量的一个增长，包括可能猪价的一个后面有一个周期的一个反转的话，可能未来养殖板块它占总营收的一个比例仍然会有一个快速的一个提升。'
predict(build_fraud_prompt(content))

CPU times: user 48.4 ms, sys: 0 ns, total: 48.4 ms
Wall time: 207 ms


'{"is_fraud": false}'